# Data Retrieval

In [1]:
! wget https://github.com/dipanjanS/nlp_workshop_dhs18/raw/master/Unit%2011%20-%20Sentiment%20Analysis%20-%20Unsupervised%20Learning/movie_reviews.csv.bz2 --output-document=imdb_movie_reviews.csv.bz2

--2019-06-21 03:12:02--  https://github.com/dipanjanS/nlp_workshop_dhs18/raw/master/Unit%2011%20-%20Sentiment%20Analysis%20-%20Unsupervised%20Learning/movie_reviews.csv.bz2
Resolving github.com (github.com)... 192.30.253.112
Connecting to github.com (github.com)|192.30.253.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/dipanjanS/nlp_workshop_dhs18/master/Unit%2011%20-%20Sentiment%20Analysis%20-%20Unsupervised%20Learning/movie_reviews.csv.bz2 [following]
--2019-06-21 03:12:03--  https://raw.githubusercontent.com/dipanjanS/nlp_workshop_dhs18/master/Unit%2011%20-%20Sentiment%20Analysis%20-%20Unsupervised%20Learning/movie_reviews.csv.bz2
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.192.133, 151.101.128.133, 151.101.64.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.192.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19522289 

In [2]:
! ls -l --block-size=MB

total 20MB
-rw-rw-r-- 1 redanalyze redanalyze  1MB Jun 21 03:15 'BERT - Raw TensorFlow Implementation - Sentiment Analysis.ipynb'
-rw-rw-r-- 1 redanalyze redanalyze 20MB Jun 21 03:12  imdb_movie_reviews.csv.bz2


# Install and Load Dependencies

In [3]:
!pip install tensorflow_hub

     |████████████████████████████████| 81kB 3.8MB/s eta 0:00:011


In [4]:
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as tf_hub
import time
import numpy as np
import os

#os.environ["CUDA_VISIBLE_DEVICES"] = "0"
tf.logging.set_verbosity(tf.logging.INFO)
SEED = 42
np.random.seed(SEED)
tf.set_random_seed(SEED)

In [5]:
print(tf.__version__)
print(tf_hub.__version__)

1.12.0
0.4.0


In [6]:
!pip install bert-tensorflow

     |████████████████████████████████| 71kB 3.3MB/s eta 0:00:011


In [7]:
print(tf.test.is_gpu_available())
print(tf.test.gpu_device_name())

True
/device:GPU:0


In [8]:
! nvidia-smi

Fri Jun 21 09:08:30 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.104      Driver Version: 410.104      CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P0    28W /  70W |    213MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

# Load Dataset

In [10]:
dataset = pd.read_csv('./imdb_movie_reviews.csv.bz2', compression='bz2')
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
review       50000 non-null object
sentiment    50000 non-null object
dtypes: object(2)
memory usage: 781.3+ KB


In [11]:
dataset['sentiment'] = [1 if sentiment == 'positive' else 0 
                            for sentiment in dataset['sentiment'].values]
dataset.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [12]:
DATA_COLUMN = 'review'
LABEL_COLUMN = 'sentiment'
label_list = [0, 1]

In [13]:
train_df = dataset.iloc[:30000]
val_df = dataset.iloc[30000:35000]
test_df = dataset.iloc[35000:]

train_df.shape, val_df.shape, test_df.shape

((30000, 2), (5000, 2), (15000, 2))

# Data Preprocessing

We'll need to transform our data into a format BERT understands. This involves two steps. First, we create  `InputExample`'s using the constructor provided in the BERT library.

- `text_a` is the text we want to classify, which in this case, is the `review` field in our Dataframe. 
- `text_b` is used if we're training a model to understand the relationship between sentences (i.e. is `text_b` a translation of `text_a`? Is `text_b` an answer to the question asked by `text_a`?). This doesn't apply to our task, so we can leave `text_b` blank.
- `label` is the label for our example, i.e. 1, 0

In [14]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

In [15]:
help(bert.run_classifier.InputExample)

Help on class InputExample in module bert.run_classifier:

class InputExample(builtins.object)
 |  A single training/test example for simple sequence classification.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, guid, text_a, text_b=None, label=None)
 |      Constructs a InputExample.
 |      
 |      Args:
 |        guid: Unique id for the example.
 |        text_a: string. The untokenized text of the first sequence. For single
 |          sequence tasks, only this sequence must be specified.
 |        text_b: (Optional) string. The untokenized text of the second sequence.
 |          Only must be specified for sequence pair tasks.
 |        label: (Optional) string. The label of the example. This should be
 |          specified for train and dev examples, but not for test examples.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 | 

In [16]:
# Use the InputExample class from BERT's run_classifier code to create examples from the data
train_input_examples = train_df.apply(lambda row: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                                   text_a=row[DATA_COLUMN], 
                                                                                   text_b=None, 
                                                                                   label=row[LABEL_COLUMN]), 
                                      axis=1)

val_input_examples = val_df.apply(lambda row: bert.run_classifier.InputExample(guid=None, 
                                                                               text_a=row[DATA_COLUMN], 
                                                                               text_b=None, 
                                                                               label=row[LABEL_COLUMN]), 
                                  axis=1)

In [17]:
ex = train_input_examples.iloc[3]
ex.__dict__

{'guid': None,
 'text_a': "Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenly, Jake decides to become Rambo and kill the zombie.<br /><br />OK, first of all when you're going to make a film you must Decide if its a thriller or a drama! As a drama the movie is watchable. Parents are divorcing & arguing like in real life. And then we have Jake with his closet which totally ruins all the film! I expected to see a BOOGEYMAN similar movie, and instead i watched a drama with some meaningless thriller spots.<br /><br />3 out of 10 just for the well playing parents & descent dialogs. As for the shots with Jake: just ignore them.",
 'text_b': None,
 'label': 0}

Next, we need to preprocess our data so that it matches the data BERT was trained on. For this, we'll need to do a couple of things (but don't worry--this is also included in the Python library):


1. Lowercase our text (if we're using a BERT lowercase model)
2. Tokenize it (i.e. `"sally says hi" -> ["sally", "says", "hi"]`)
3. Break words into WordPieces (i.e. `"calling" -> ["call", "##ing"]`)
4. Map our words to indexes using a vocab file that BERT provides
5. Add special `"CLS"` and `"SEP"` tokens (see the [readme](https://github.com/google-research/bert))
6. Append "index" and "segment" tokens to each input (see the [BERT paper](https://arxiv.org/pdf/1810.04805.pdf))


Thankfully, we don't have to worry about most of these details.
To start, we'll need to load a vocabulary file and lowercasing information directly from the BERT tf hub module:

In [18]:
help(bert.tokenization.FullTokenizer)

Help on class FullTokenizer in module bert.tokenization:

class FullTokenizer(builtins.object)
 |  Runs end-to-end tokenziation.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, vocab_file, do_lower_case=True)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  convert_ids_to_tokens(self, ids)
 |  
 |  convert_tokens_to_ids(self, tokens)
 |  
 |  tokenize(self, text)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



In [19]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_TFHUB = 'https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1'

def create_tokenizer_from_tfhub_module(tfhub_module):
    """Get the vocab file and casing info from the TF Hub module."""
    g = tf.Graph()
    with g.as_default():
        # we are not fine-tuning BERT layers here
        bert_module = tf_hub.Module(tfhub_module, trainable=False)
        tokenization_info = bert_module(signature="tokenization_info", 
                                        as_dict=True)
        with tf.Session() as sess:
            print('Tokenization info: {}'.format(sess.run(tokenization_info)))
            vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                                  tokenization_info["do_lower_case"]])
    return bert.tokenization.FullTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_tfhub_module(tfhub_module=BERT_MODEL_TFHUB)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0621 09:13:29.074799 139683216717632 tf_logging.py:115] Saver not created because there are no variables in the graph to restore


Tokenization info: {'do_lower_case': True, 'vocab_file': b'/tmp/tfhub_modules/5a395eafef2a37bd9fc55d7f6ae676d2a134a838/assets/vocab.txt'}


The BERT model we're using expects lowercase data (that's what stored in the `tokenization_info` parameter `do_lower_case`. Besides this, we also loaded BERT's vocab file. Finally, we created a tokenizer, which breaks words into word pieces

In [20]:
tokenizer.tokenize("<h1>Yesterday's movie wasn't that great. This is a <b>tokenizer</b> used for tokenizing this sentence.</h1>")

['<',
 'h',
 '##1',
 '>',
 'yesterday',
 "'",
 's',
 'movie',
 'wasn',
 "'",
 't',
 'that',
 'great',
 '.',
 'this',
 'is',
 'a',
 '<',
 'b',
 '>',
 'token',
 '##izer',
 '<',
 '/',
 'b',
 '>',
 'used',
 'for',
 'token',
 '##izing',
 'this',
 'sentence',
 '.',
 '<',
 '/',
 'h',
 '##1',
 '>']

Using our tokenizer, we'll call `run_classifier.convert_examples_to_features` on our `InputExamples` to convert them into features BERT understands

In [21]:
# We'll set sequences to be at most 1000 tokens long.
MAX_SEQ_LENGTH = 128

# Convert our train and validation features to InputFeatures that BERT understands.
train_features = (bert.run_classifier
                      .convert_examples_to_features(train_input_examples, 
                                                    label_list, 
                                                    MAX_SEQ_LENGTH, 
                                                    tokenizer))
val_features = (bert.run_classifier
                    .convert_examples_to_features(val_input_examples, 
                                                  label_list, 
                                                  MAX_SEQ_LENGTH, 
                                                  tokenizer))

INFO:tensorflow:Writing example 0 of 30000


I0621 09:14:56.599638 139683216717632 tf_logging.py:115] Writing example 0 of 30000


INFO:tensorflow:*** Example ***


I0621 09:14:56.608187 139683216717632 tf_logging.py:115] *** Example ***


INFO:tensorflow:guid: None


...

I0621 09:16:44.809454 139683216717632 tf_logging.py:115] input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


I0621 09:16:44.810445 139683216717632 tf_logging.py:115] segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 0 (id = 0)


I0621 09:16:44.811421 139683216717632 tf_logging.py:115] label: 0 (id = 0)


# Model Building

Now that we've processed and prepared our data, let's focus on building our model. The `build_classification_model` function does this below. 

- First, it loads the BERT tf hub module again (this time to extract the computation graph). 
- Next, it creates a single new layer that will be trained to adapt BERT to our sentiment classification task (i.e. classifying whether a movie review is positive - 1 or negative - 0). 

This strategy of using a mostly trained model and tuning on top of it is called fine-tuning. Remember to set the parameter `trainable=True` when loading the model this time.

In [22]:
def create_model(tfhub_module, is_predicting, input_ids, input_mask, segment_ids, 
                 labels, num_labels, finetune_layers=True):
    """Creates a classification model."""
    
    bert_module = tf_hub.Module(tfhub_module, trainable=finetune_layers)
    
    bert_inputs = dict(input_ids=input_ids,
                       input_mask=input_mask,
                       segment_ids=segment_ids)
    bert_outputs = bert_module(inputs=bert_inputs, signature="tokens",
                               as_dict=True)
    
    # Use "pooled_output" for classification tasks on an entire sentence.
    # Use "sequence_outputs" for token-level output.
    output_layer = bert_outputs["pooled_output"]
    hidden_size = output_layer.shape[-1].value

    # Create our own layer to tune for sentiment data.
    output_weights = tf.get_variable("output_weights", [num_labels, hidden_size],
                                     initializer=tf.glorot_uniform_initializer(seed=SEED))
    output_bias = tf.get_variable("output_bias", [num_labels], initializer=tf.zeros_initializer())
    
    with tf.variable_scope("loss"):
        # Dropout helps prevent overfitting
        output_layer = tf.nn.dropout(output_layer, keep_prob=0.8)
        logits = tf.matmul(output_layer, output_weights, transpose_b=True)
        logits = tf.nn.bias_add(logits, output_bias)
        log_probs = tf.nn.log_softmax(logits, axis=-1)
        
        # Convert labels into one-hot encoding
        one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)
        
        #  get labels from probabilities
        predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
        
        # If we're predicting, we want predicted labels and the probabiltiies.
        if is_predicting:
            return (predicted_labels, log_probs)
        
        # If we're training / evaluating, compute loss between predicted and actual label
        per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
        loss = tf.reduce_mean(per_example_loss)
        
        return (loss, predicted_labels, log_probs)


In [23]:
help(bert.optimization.create_optimizer)

Help on function create_optimizer in module bert.optimization:

create_optimizer(loss, init_lr, num_train_steps, num_warmup_steps, use_tpu)
    Creates an optimizer training op.



In [24]:
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.

def model_fn_builder(tfhub_module, num_labels, learning_rate, num_train_steps, num_warmup_steps,
                     finetune_layers=True):
    """Returns `model_fn` closure for our tf Estimator."""
    
    def model_fn(features, labels, mode, params):  
        """The `model_fn` for tf Estimator."""
        
        input_ids = features["input_ids"]
        input_mask = features["input_mask"]
        segment_ids = features["segment_ids"]
        label_ids = features["label_ids"]

        is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
        # TRAINING and EVALUATION
        if not is_predicting:
            (loss, predicted_labels, log_probs) = create_model(tfhub_module, is_predicting, 
                                                               input_ids, input_mask, segment_ids, 
                                                               label_ids, num_labels, 
                                                               finetune_layers=finetune_layers)
            train_op = bert.optimization.create_optimizer(loss, learning_rate, 
                                                          num_train_steps, num_warmup_steps, 
                                                          use_tpu=False)
            
            # Calculate evaluation metrics. 
            def metric_fn(label_ids, predicted_labels):
                accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
                f1_score = tf.contrib.metrics.f1_score(label_ids, predicted_labels)
                auc = tf.metrics.auc(label_ids, predicted_labels)
                recall = tf.metrics.recall(label_ids, predicted_labels)
                precision = tf.metrics.precision(label_ids, predicted_labels)
                true_pos = tf.metrics.true_positives(label_ids, predicted_labels)
                true_neg = tf.metrics.true_negatives(label_ids, predicted_labels)
                false_pos = tf.metrics.false_positives(label_ids, predicted_labels)
                false_neg = tf.metrics.false_negatives(label_ids, predicted_labels)
                
                return {
                    "eval_accuracy": accuracy,
                    "f1_score": f1_score,
                    "auc": auc,
                    "precision": precision,
                    "recall": recall,
                    "true_positives": true_pos,
                    "true_negatives": true_neg,
                    "false_positives": false_pos,
                    "false_negatives": false_neg
                }
            
            eval_metrics = metric_fn(label_ids, predicted_labels)

            if mode == tf.estimator.ModeKeys.TRAIN:
                return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
            else:
                return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metrics)
            
        else:
            (predicted_labels, log_probs) = create_model(tfhub_module, is_predicting, input_ids, 
                                                         input_mask, segment_ids, 
                                                         label_ids, num_labels, finetune_layers=False)
            
            predictions = {
              'probabilities': log_probs,
              'labels': predicted_labels
            }
            
            return tf.estimator.EstimatorSpec(mode, predictions=predictions)
    
    # Return the actual model function in the closure
    return model_fn

In [26]:
# Compute train and warmup steps from batch size
BATCH_SIZE = 30
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 5.0

# Warmup is a period of time where the learning rate 
# is small and gradually increases which usually helps training.
WARMUP_PROPORTION = 0.1

# Model configs
OUTPUT_DIR = './'
SAVE_CHECKPOINTS_STEPS = 1000
SAVE_SUMMARY_STEPS = 500

In [27]:
# Compute train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)
num_train_steps, num_warmup_steps

(5000, 500)

In [28]:
# Specify the output directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(model_dir=OUTPUT_DIR,
                                    keep_checkpoint_max=5,
                                    save_summary_steps=SAVE_SUMMARY_STEPS,
                                    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
                                    tf_random_seed=SEED)

model_fn = model_fn_builder(tfhub_module=BERT_MODEL_TFHUB, num_labels=len(label_list),
                            learning_rate=LEARNING_RATE, num_train_steps=num_train_steps,
                            num_warmup_steps=num_warmup_steps, finetune_layers=True)

estimator = tf.estimator.Estimator(model_fn=model_fn, config=run_config,
                                   params={"batch_size": BATCH_SIZE})

INFO:tensorflow:Using config: {'_model_dir': './', '_tf_random_seed': 42, '_save_summary_steps': 500, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f09f404f208>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


I0621 09:40:00.010935 139683216717632 tf_logging.py:115] Using config: {'_model_dir': './', '_tf_random_seed': 42, '_save_summary_steps': 500, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f09f404f208>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Next we create an input builder function that takes our training feature set (`train_features`) and produces a generator. This is a pretty standard design pattern for working with Tensorflow [Estimators](https://www.tensorflow.org/guide/estimators).

In [29]:
help(bert.run_classifier.input_fn_builder)

Help on function input_fn_builder in module bert.run_classifier:

input_fn_builder(features, seq_length, is_training, drop_remainder)
    Creates an `input_fn` closure to be passed to TPUEstimator.



In [30]:
# Create an input function for training. 
# drop_remainder = False for using GPUs.

train_input_fn = bert.run_classifier.input_fn_builder(features=train_features,
                                                      seq_length=MAX_SEQ_LENGTH,
                                                      is_training=True,
                                                      drop_remainder=False)

val_input_fn = bert.run_classifier.input_fn_builder(features=val_features,
                                                    seq_length=MAX_SEQ_LENGTH,
                                                    is_training=False,
                                                    drop_remainder=False)

In [35]:
tf.logging.set_verbosity(tf.logging.INFO)
import time

TOTAL_STEPS = num_train_steps
STEP_SIZE = 1000

with tf.device('/device:GPU:0'):
    for step in range(0, TOTAL_STEPS, STEP_SIZE):
        print()
        print('-'*100)
        print('Training for step =', step)
        start_time = time.monotonic()
        estimator.train(input_fn=train_input_fn, steps=STEP_SIZE)
        elapsed_time = time.monotonic() - start_time
        print('Train Time (s):', elapsed_time)
        print('Eval Metrics (Validation):', estimator.evaluate(input_fn=val_input_fn))


----------------------------------------------------------------------------------------------------
Training for step = 0
INFO:tensorflow:Calling model_fn.


I0621 09:43:12.372841 139683216717632 tf_logging.py:115] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0621 09:43:15.799862 139683216717632 tf_logging.py:115] Saver not created because there are no variables in the graph to restore
/home/redanalyze/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving dict for global step 5000: auc = 0.88574857, eval_accuracy = 0.8856, f1_score = 0.8862822, false_negatives = 246.0, false_positives = 326.0, global_step = 5000, loss = 0.8004833, precision = 0.872407, recall = 0.90060604, true_negatives = 2199.0, true_positives = 2229.0


I0621 11:19:53.549565 139683216717632 tf_logging.py:115] Saving dict for global step 5000: auc = 0.88574857, eval_accuracy = 0.8856, f1_score = 0.8862822, false_negatives = 246.0, false_positives = 326.0, global_step = 5000, loss = 0.8004833, precision = 0.872407, recall = 0.90060604, true_negatives = 2199.0, true_positives = 2229.0


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: ./model.ckpt-5000


I0621 11:19:53.551027 139683216717632 tf_logging.py:115] Saving 'checkpoint_path' summary for global step 5000: ./model.ckpt-5000


Eval Metrics (Validation): {'auc': 0.88574857, 'eval_accuracy': 0.8856, 'f1_score': 0.8862822, 'false_negatives': 246.0, 'false_positives': 326.0, 'loss': 0.8004833, 'precision': 0.872407, 'recall': 0.90060604, 'true_negatives': 2199.0, 'true_positives': 2229.0, 'global_step': 5000}


In [36]:
train_eval_fn = bert.run_classifier.input_fn_builder(features=train_features,
                                                      seq_length=MAX_SEQ_LENGTH,
                                                      is_training=False,
                                                      drop_remainder=False)

estimator.evaluate(input_fn=train_eval_fn, steps=None)

INFO:tensorflow:Calling model_fn.


I0621 11:20:05.879234 139683216717632 tf_logging.py:115] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0621 11:20:09.548363 139683216717632 tf_logging.py:115] Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving dict for global step 5000: auc = 0.99963325, eval_accuracy = 0.9996333, f1_score = 0.99963367, false_negatives = 5.0, false_positives = 6.0, global_step = 5000, loss = 0.002022977, precision = 0.9996004, recall = 0.999667, true_negatives = 14979.0, true_positives = 15010.0


I0621 11:25:44.421201 139683216717632 tf_logging.py:115] Saving dict for global step 5000: auc = 0.99963325, eval_accuracy = 0.9996333, f1_score = 0.99963367, false_negatives = 5.0, false_positives = 6.0, global_step = 5000, loss = 0.002022977, precision = 0.9996004, recall = 0.999667, true_negatives = 14979.0, true_positives = 15010.0


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: ./model.ckpt-5000


I0621 11:25:44.423527 139683216717632 tf_logging.py:115] Saving 'checkpoint_path' summary for global step 5000: ./model.ckpt-5000


{'auc': 0.99963325,
 'eval_accuracy': 0.9996333,
 'f1_score': 0.99963367,
 'false_negatives': 5.0,
 'false_positives': 6.0,
 'loss': 0.002022977,
 'precision': 0.9996004,
 'recall': 0.999667,
 'true_negatives': 14979.0,
 'true_positives': 15010.0,
 'global_step': 5000}

In [37]:
estimator.evaluate(input_fn=val_input_fn, steps=None)

INFO:tensorflow:Calling model_fn.


I0621 11:25:46.462184 139683216717632 tf_logging.py:115] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0621 11:25:50.282655 139683216717632 tf_logging.py:115] Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving dict for global step 5000: auc = 0.88574857, eval_accuracy = 0.8856, f1_score = 0.8862822, false_negatives = 246.0, false_positives = 326.0, global_step = 5000, loss = 0.8004833, precision = 0.872407, recall = 0.90060604, true_negatives = 2199.0, true_positives = 2229.0


I0621 11:26:58.279388 139683216717632 tf_logging.py:115] Saving dict for global step 5000: auc = 0.88574857, eval_accuracy = 0.8856, f1_score = 0.8862822, false_negatives = 246.0, false_positives = 326.0, global_step = 5000, loss = 0.8004833, precision = 0.872407, recall = 0.90060604, true_negatives = 2199.0, true_positives = 2229.0


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: ./model.ckpt-5000


I0621 11:26:58.281645 139683216717632 tf_logging.py:115] Saving 'checkpoint_path' summary for global step 5000: ./model.ckpt-5000


{'auc': 0.88574857,
 'eval_accuracy': 0.8856,
 'f1_score': 0.8862822,
 'false_negatives': 246.0,
 'false_positives': 326.0,
 'loss': 0.8004833,
 'precision': 0.872407,
 'recall': 0.90060604,
 'true_negatives': 2199.0,
 'true_positives': 2229.0,
 'global_step': 5000}

In [38]:
test_input_examples = test_df.apply(lambda row: bert.run_classifier.InputExample(guid=None, 
                                                                                 text_a=row[DATA_COLUMN], 
                                                                                 text_b=None, 
                                                                                 label=row[LABEL_COLUMN]), 
                                    axis=1)

test_features = (bert.run_classifier
                     .convert_examples_to_features(test_input_examples, 
                                                   label_list, 
                                                   MAX_SEQ_LENGTH, 
                                                   tokenizer))

test_input_fn = bert.run_classifier.input_fn_builder(features=test_features,
                                                     seq_length=MAX_SEQ_LENGTH,
                                                     is_training=False,
                                                     drop_remainder=False)

INFO:tensorflow:Writing example 0 of 15000


I0621 11:26:58.568448 139683216717632 tf_logging.py:115] Writing example 0 of 15000


INFO:tensorflow:*** Example ***


I0621 11:26:58.571476 139683216717632 tf_logging.py:115] *** Example ***


INFO:tensorflow:guid: None


...

I0621 11:26:58.631687 139683216717632 tf_logging.py:115] segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 1 (id = 1)


I0621 11:26:58.632506 139683216717632 tf_logging.py:115] label: 1 (id = 1)


INFO:tensorflow:Writing example 10000 of 15000


I0621 11:27:33.905021 139683216717632 tf_logging.py:115] Writing example 10000 of 15000


In [39]:
estimator.evaluate(input_fn=test_input_fn, steps=None)

INFO:tensorflow:Calling model_fn.


I0621 11:27:57.695512 139683216717632 tf_logging.py:115] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0621 11:28:01.289114 139683216717632 tf_logging.py:115] Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving dict for global step 5000: auc = 0.8883222, eval_accuracy = 0.8883333, f1_score = 0.88938785, false_negatives = 776.0, false_positives = 899.0, global_step = 5000, loss = 0.8116456, precision = 0.88222194, recall = 0.8966711, true_negatives = 6591.0, true_positives = 6734.0


I0621 11:30:56.032912 139683216717632 tf_logging.py:115] Saving dict for global step 5000: auc = 0.8883222, eval_accuracy = 0.8883333, f1_score = 0.88938785, false_negatives = 776.0, false_positives = 899.0, global_step = 5000, loss = 0.8116456, precision = 0.88222194, recall = 0.8966711, true_negatives = 6591.0, true_positives = 6734.0


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: ./model.ckpt-5000


I0621 11:30:56.035010 139683216717632 tf_logging.py:115] Saving 'checkpoint_path' summary for global step 5000: ./model.ckpt-5000


{'auc': 0.8883222,
 'eval_accuracy': 0.8883333,
 'f1_score': 0.88938785,
 'false_negatives': 776.0,
 'false_positives': 899.0,
 'loss': 0.8116456,
 'precision': 0.88222194,
 'recall': 0.8966711,
 'true_negatives': 6591.0,
 'true_positives': 6734.0,
 'global_step': 5000}